In [2]:
import pandas as pd
from datetime import datetime
from ydata_profiling import ProfileReport

Parte 1:
--------
    - Crear una columna nueva (ejemplo nombre tipo_ocupacion) e indicar la cantidad de personas con y sin ocupación.
    
    En base al dataset provisto cree una columna 'empleo_sl' que se base en la situacion laboral provista.
    Sin embargo, la mitad de los registros son: 'Sin_situacion_declarada'.
    Por lo que en base a los ingresos rellene esa categoría en la columna 'empleo_i'

### Cargamos el dataset

In [3]:
df1 = pd.read_csv('./raw data/Tabla1_ejercicio_N3.csv', sep=';')                            # cargamos el dataset
df1 = df1.drop_duplicates()                                                                 # eliminar duplicados

print(df1, '\n')                                                                            # visualizamos el dataset
print(df1.dtypes, '\n')                                                                     # verificar los tipos de datos
print(df1.isnull().sum(), '\n')                                                             # cantidad de null * col

          pers_id sexo  edad      ingreso            situacion_lab
0     71906556521    F    47  35926.28125  Sin_situacion_declarada
1     73861260912    F    41  16231.12207  Sin_situacion_declarada
2     24939194469    F    40  25068.96484     Relacion_dependencia
3     21033369421    F    52  30239.54883                 Jubilado
4     45334163605    F    46  12654.58203     Relacion_dependencia
...           ...  ...   ...          ...                      ...
4995  77317158601    M    55  22806.19336     Relacion_dependencia
4996   7865860477    M    42  34291.91016     Relacion_dependencia
4997   2875330898    M    55  19692.85938  Sin_situacion_declarada
4998  57646643429    M    33  37955.50781     Relacion_dependencia
4999  92370836565    M    41  92338.20313     Relacion_dependencia

[4853 rows x 5 columns] 

pers_id            int64
sexo              object
edad               int64
ingreso          float64
situacion_lab     object
dtype: object 

pers_id          0
sexo     

### Creamos un EDA con pandas-profiling

In [4]:
profile = ProfileReport(df1)                                                                # generamos un EDA con pandas_profiling
profile.to_file("EDA.html")                                                                 # exportamos el reporte en HTML

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Conclusiones:
-------------
    El dataset no contenía valores nulos ni valores duplicados...

    - La empresa tiene el doble de clientes femeninas que masculinos.
    - La mayoria de sus clientes tienen mas de 40 años, aproximadamente del doble de sus clientes son mayores de 40 años en comparacion con los menores de 40 años. Aunque disminuyen coinsiderablemente luego de los 60 años.
    - El minimo ingreso es 5 mil y el maximo 850 mil. Lo que comprueba que hay clientes que tienen ingresos mas no trabajo. y la media es de 48 mil.
    - Hay una correlacion considerable entre sexo y edad. puede deverse a que los clientes de cierta edad sean hombres mientras que los de otra edad sean mujeres.

### Empleo según la columna situacion_lab

In [6]:
df1['empleo_sl'] = df1['situacion_lab']                                                     # duplicamos 'situacion_lab' en 'empleo_sl' que contendra 0 y 1 

reemplazos = {                                                                              # creamos la variable reemplazos con las distintas variables a reemplazar por 0(desempleo) o 1(empleo)
    'Desempleado_o_plansocial': 0,                                                          # Desempleado_o_plansocial, como dice la palabra es desempleo
    'Jubilado': 0,                                                                          # Jubilado, no trabaja, por ende desempleo.
    'Autonomo_Monotributista': 1,                                                           # Autonomo o Monotributista, trabajan incluso siendo emprendedores, es trabajo
    'Relacion_dependencia': 1,                                                              # Relacion_dependencia, empleo por definicion
    'Sin_situacion_declarada': None                                                         # Sin_situacion_declarada, lo seteamos como null para rellenar en base a los ingresos
}

df1['empleo_sl'] = df1['empleo_sl'].replace(reemplazos)                                     # reemplazamos las categorias por empleo o desempleo
df1['empleo_sl'] = pd.to_numeric(df1['empleo_sl'], errors='coerce').astype(pd.Int64Dtype()) # Convertir la columna a tipo entero ignorando nulos

### Empleo según la columna ingresos

In [7]:
median = df1['ingreso'].median()                                                            # obtenemos la mediana de la columna 'ingreso'
print('El ingreso medio es de: ', int(median), '\n')                                        # imprimimos el ingreso medio                   

def Empleo_i(i):                                                                            # creamos una función para transformar los valores en 'empleo_i'
    if i < median / 2:
        return 0
    else:
        return 1

df1['empleo_i'] = df1.apply(lambda row: Empleo_i(row['ingreso']) if pd.isnull(row['empleo_sl']) else row['empleo_sl'], axis=1)  # aplicamos la función Empleo_i a la columna 'empleo_i' solo si los valores en 'empleo_sl' son nulos


El ingreso medio es de:  33383 



### Cantidad de clientes con empleo y desempleados

In [8]:
cantidad_empleo = (df1['empleo_i'] == 1).sum()                                              # contamos los empleados en empleo_i
cantidad_desempleo = (df1['empleo_i'] == 0).sum()                                           # contamos los desempleados en empleo_i
print("Cantidad de clientes con empleo:", cantidad_empleo)                                  # imprimimos la cantidad de empleados
print("Cantidad de clientes desempleados:", cantidad_desempleo, '\n')                       # imprimimos la cantidad de desempleados

Cantidad de clientes con empleo: 4038
Cantidad de clientes desempleados: 815 



### Creamos la columna rango_etario

In [9]:
def calcular_rango_etario(edad):                                                            # creamos una función para calcular el rango etario por décadas
    return f"{(edad // 10) * 10}-{((edad // 10) * 10) + 9}"                                 # redondeamos la edad a la década inferior
df1['rango_etario'] = df1['edad'].apply(calcular_rango_etario)                              # aplicamos la función

### Exportamos el dataset limpio

In [10]:
df1.set_index('pers_id', inplace=True)                                                      # establecemos la columna 'Id' como índice
df1.to_csv('./data/Tabla1.csv')                                                             # exportamos el dataset en csv a /data
print(df1, '\n')                                                                            # imprimimos el dataset

            sexo  edad      ingreso            situacion_lab  empleo_sl  \
pers_id                                                                   
71906556521    F    47  35926.28125  Sin_situacion_declarada       <NA>   
73861260912    F    41  16231.12207  Sin_situacion_declarada       <NA>   
24939194469    F    40  25068.96484     Relacion_dependencia          1   
21033369421    F    52  30239.54883                 Jubilado          0   
45334163605    F    46  12654.58203     Relacion_dependencia          1   
...          ...   ...          ...                      ...        ...   
77317158601    M    55  22806.19336     Relacion_dependencia          1   
7865860477     M    42  34291.91016     Relacion_dependencia          1   
2875330898     M    55  19692.85938  Sin_situacion_declarada       <NA>   
57646643429    M    33  37955.50781     Relacion_dependencia          1   
92370836565    M    41  92338.20313     Relacion_dependencia          1   

             empleo_i ra

Parte 2:
--------
    Teniendo en cuenta el periodo más reciente disponible para cada persona, crear una función que reciba como inputs:
        - Parámetro 1: datos input.
        - Parámetro 2: estadístico que se va a calcular.
        - Parámetro 3: variable a la que se va a aplicar el estadístico.
        - Parámetro 4: variable con la cual se va a agrupar.
    
    Y calcular:
        - Score promedio y mediana según categoría ocupacional.
        - Ingreso y Score máximo y promedio según sexo.

### Cargar el 2do dataset

In [11]:
df2 = pd.read_csv('./raw data/Tabla2_ejercicio_N3.csv', sep=';')                            # cargamos el dataset
df2 = df2.drop_duplicates()                                                                 # eliminamos duplicados

print(df2, '\n')                                                                            # visualizamos el dataset
print(df2.dtypes, '\n')                                                                     # verificar los tipos de datos
print(df2.isnull().sum(), '\n')                                                             # cantidad de null * col

           pers_id  periodo  score
0         27817725   201906    814
1         37570691   201906    597
2         40181240   201906    251
3         52105045   201906    617
4         63502595   201906    392
...            ...      ...    ...
14991  99744055227   201912    943
14992  99788780024   201912    513
14993  99897294254   201912    904
14994  99938763350   201912      1
14996  99958693785   201912    642

[14721 rows x 3 columns] 

pers_id    int64
periodo    int64
score      int64
dtype: object 

pers_id    0
periodo    0
score      0
dtype: int64 



### Limpiar el 2do dataset

In [12]:
df2['periodo'] = df2['periodo'].astype(str)                                                 # convertimos 'periodo' en str
df2['periodo'] = df2['periodo'].apply(lambda x: datetime.strptime(x, '%Y%m'))               # convertimos 'periodo' a datetime

df2 = df2.sort_values(by=['pers_id', 'periodo'], ascending=[True, False])                   # ordenamos el df por 'pers_id' y 'periodo' en orden descendente
df2 = df2.groupby('pers_id').first().reset_index()                                          # mantenemos solo la primera fila para cada 'pers_id'

### Exportamos el 2do dataset

In [13]:
df2.set_index('pers_id', inplace=True)                                                      # establecemos la columna 'Id' como índice
df2.to_csv('./data/Tabla2.csv')                                                             # exportamos el dataset en csv a /data
print(df2, '\n')                                                                            # imprimimos el dataset

               periodo  score
pers_id                      
27817725    2019-12-01    814
37570691    2019-12-01    597
40181240    2019-12-01    251
52105045    2019-12-01    617
63502595    2019-12-01    392
...                ...    ...
99744055227 2019-12-01    943
99788780024 2019-12-01    513
99897294254 2019-12-01    904
99938763350 2019-12-01      1
99958693785 2019-12-01    642

[4853 rows x 2 columns] 



### Unimos los 2 datasets

In [14]:
df = pd.merge(df1, df2, on='pers_id', how='right')                                          # unimos los df con un 'right join' debido a que se nos pide: 'Teniendo en cuenta el periodo más reciente disponible'. Es decir, teniendo en cuenta el periodo.
df = df.drop('empleo_sl', axis=1)                                                           # eliminamos la columna 'empleo_sl'
df.rename(columns={'empleo_i': 'empleo'}, inplace=True)                                     # renombramos la columna 'empleo_i' a 'empleo'
df.to_csv('./data/Tabla.csv')                                                               # exportamos el dataset en csv a /data
print(df, '\n')                                                                             # imprimimos el dataset

            sexo  edad      ingreso            situacion_lab  empleo  \
pers_id                                                                
27817725       F    40  10622.45898  Sin_situacion_declarada       0   
37570691       M    24  43974.76563     Relacion_dependencia       1   
40181240       F    51  27217.44727  Autonomo_Monotributista       1   
52105045       F    56  55331.26563  Autonomo_Monotributista       1   
63502595       F    50  33757.90625     Relacion_dependencia       1   
...          ...   ...          ...                      ...     ...   
99744055227    F    56  53152.60938                 Jubilado       0   
99788780024    F    56  61848.82031  Autonomo_Monotributista       1   
99897294254    F    44  47892.31250     Relacion_dependencia       1   
99938763350    F    52  12823.33105  Sin_situacion_declarada       0   
99958693785    F    29  25437.70898  Autonomo_Monotributista       1   

            rango_etario    periodo  score  
pers_id           

### Creamos un 'dashboard' con pandas-profiling

In [15]:
profile = ProfileReport(df)                                                                 # generamos un dashboard con pandas_profiling
profile.to_file("./docs/conclusiones.html")                                                 # exportamos el reporte en HTML

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Crear la funcion solicitada

In [16]:
def Calcular_Estadisticas(df, est, var1, var2):                                             # creamos la funcion que calculara los metodos estadisticos
    """
    funcion capaz de calcular metodos estadisticos por columna y valores agrupados en un dataset.

    Args:
        df (DataFrame): DataFrame con los datos.
        est (str): estadistico a calular ['mean', 'median', 'sum', 'std', 'var', 'count', 'sem', 'max', 'min'].
        var1 (str): variable a la que se va a aplicar el estadistico.
        var2 (str): variable con la cual se va a agrupar.
        bins (int or list of int, optional): Parámetro para binning (por ejemplo, para agrupar edades por décadas).

    Returns:
        DataFrame: Resultados de las estadísticas calculadas.
    """
    if est not in ['mean', 'median', 'sum', 'std', 'var', 'count', 'sem', 'max', 'min']:                                        # si el parametro 'est' no un estadistico correcto, retorna error
        raise ValueError("Estadístico debe ser: 'mean', 'median', 'sum', 'std', 'var', 'count', 'sem', 'max' o 'min'")

    if (var1 and var2) not in ['sexo', 'edad', 'rango_etario', 'ingreso', 'situacion_lab', 'empleo', 'periodo', 'score']:                       # si las variables no son las del dataset, retorna error
        raise ValueError("Las variables deben ser: 'sexo', 'edad', 'ingreso', 'situacion_lab', 'empleo', 'periodo' o 'score'")

    if var1 == var2:                                                                        # si las variables son iguales, retorna error
        raise ValueError("Las variables no pueden ser iguales")

    if est == 'max':                                                                        # calculamos el maximo
        grouped_data = df.groupby(var2)[var1].max().reset_index()
    elif est == 'min':                                                                      # calculamos el minimo
        grouped_data = df.groupby(var2)[var1].min().reset_index()
    else:                                                                                   # calculamos el estadistico seleccionado
        grouped_data = df.groupby(var2)[var1].agg([est]).reset_index()
    
    return grouped_data

### Obtener resultados

In [17]:
result1 = Calcular_Estadisticas(df, 'mean', 'score', 'empleo')                              # calculamos el score promedio según empleo
result2 = Calcular_Estadisticas(df, 'median', 'score', 'empleo')                            # calculamos el score medio según empleo
result3 = Calcular_Estadisticas(df, 'max', 'ingreso', 'sexo')                               # calculamos el ingreso maximo según sexo
result4 = Calcular_Estadisticas(df, 'mean', 'ingreso', 'sexo')                              # calculamos el ingreso promedio según sexo
result5 = Calcular_Estadisticas(df, 'max', 'score', 'sexo')                                 # calculamos el score maximo según sexo
result6 = Calcular_Estadisticas(df, 'mean', 'score', 'sexo')                                # calculamos el score promedio según sexo
result7 = Calcular_Estadisticas(df, 'mean', 'edad', 'sexo')                                 # calculamos la edad promedio según sexo
result8 = Calcular_Estadisticas(df, 'mean', 'score', 'rango_etario')                        # calculamos el score promedio según rango_etario

print('score promedio según empleo:')
print(result1, '\n')                                                                        # imprimimos el score promedio según empleo
print('score medio según empleo:')
print(result2, '\n')                                                                        # imprimimos el score medio según empleo
print('ingreso maximo según sexo:')
print(result3, '\n')                                                                        # imprimimos el ingreso maximo según sexo
print('ingreso promedio según sexo:')
print(result4, '\n')                                                                        # imprimimos el ingreso promedio según sexo
print('score maximo según sexo:')
print(result5, '\n')                                                                        # imprimimos el score maximo según sexo
print('score promedio según sexo:')
print(result6, '\n')                                                                        # imprimimos el score promedio según sexo
print('edad promedio según sexo:')
print(result7, '\n')                                                                        # imprimimos la edad promedio según sexo
print('score promedio según rango_etario:')
print(result8, '\n')                                                                        # imprimimos el score promedio según rango_etario

score promedio según empleo:
   empleo        mean
0       0  566.711656
1       1  696.622338 

score medio según empleo:
   empleo  median
0       0   620.0
1       1   745.0 

ingreso maximo según sexo:
  sexo     ingreso
0    F  850432.250
1    M  669146.625 

ingreso promedio según sexo:
  sexo          mean
0    F  44848.931498
1    M  55543.766640 

score maximo según sexo:
  sexo  score
0    F    965
1    M   1011 

score promedio según sexo:
  sexo        mean
0    F  616.837757
1    M  804.382667 

edad promedio según sexo:
  sexo       mean
0    F  47.708619
1    M  42.066667 

score promedio según rango_etario:
  rango_etario        mean
0        10-19  839.000000
1        20-29  787.588235
2        30-39  782.147105
3        40-49  627.281853
4        50-59  655.634387
5        60-69  748.825503 

